## Import stuff

In [2]:
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tensorflow.keras import layers
import shutil
import random
from PIL import Image
import numpy as np
import logging
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
import torch 
import torch.jit
import torch.nn as nn 
import torchvision
from torchvision import transforms, datasets
import math
logger = tf.get_logger()
logger.setLevel(logging.ERROR)
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

print(tf.__version__)

from tensorflow.python.client import device_lib
print("tensorflow doesn't detects GPU: ")
print(device_lib.list_local_devices() )
print(tf.test.gpu_device_name())
print(tf.config.list_physical_devices())

print("torch detects GPU: " , torch.cuda.is_available())


2.3.0
tensorflow doesn't detects GPU: 
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 5883077918976011752
]

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]
torch detects GPU:  True


## Load training data from csv


In [5]:
patients = pd.read_csv("train_features.csv")


#get the mean for each column
mean_values = patients.mean()


# go through 12 rows at a time
for i in range(0, len(patients), 12):
    # get the 12 rows
    patient = patients.iloc[i:i+12]
    #for each column, use linear interpolation to replace the nan values
    for col in patient.columns:
        patient[col] = patient[col].interpolate(method='linear')
        mean = patient[col].mean()
        if math.isnan(mean):
            patient[col] = mean_values[col]
        mean = patient[col].mean()
        patient[col] = patient[col].fillna(mean)



    if i == 2*12:
        print(i)
        break
    # break



#save the dataframe
patients.to_csv("train_features_fixed.csv", index=False, float_format='%.3f',na_rep='nan')




<ipython-input-5-73ea031df8f4>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  patient[col] = patient[col].interpolate(method='linear')
<ipython-input-5-73ea031df8f4>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  patient[col] = patient[col].fillna(mean)
<ipython-input-5-73ea031df8f4>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pand

24


In [30]:
patients = pd.read_csv("train_features_fixed.csv")

#normalize each column except the first 2
patients2 = patients.iloc[:,2:]
patients2 = patients2.apply(lambda x: (x - x.min()) / (x.max() - x.min()))

#apply the changes to the original
patients.iloc[:,2:] = patients2


#save the dataframe
patients.to_csv("train_features_normalized.csv", index=False, float_format='%.3f',na_rep='nan')